# Нейроэволюционный алгоритм

## Загружаем и подготавливаем данные для работы

### Строки, не несущие информацию были удалены

In [ ]:
import datatable as dt
data = dt.fread('cancer1.dt')
del data[0:7,:]

## Преробразуем данные в формат pandas.DataFrame 

In [ ]:
import pandas as pd
df = pd.DataFrame(data.to_pandas())
df

## Разделяем данные на входные и требуемые выходные значения

In [ ]:
outputs = df[['C9','C10']] \
                    .iloc[:,0]\
                    .astype('int64')

outputs

In [ ]:
inputs = df.iloc[:, 0:9].copy().astype('float64')
inputs

## Разбиваем выборку на тренировочную и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, train_size = 0.8, random_state = 3)

In [ ]:
import numpy as np
from NeuralNetwork import NeuralNetwork

$BLX-{\alpha}$ кроссинговер.

In [ ]:
def crossbreeding(nn_parent_1, nn_parent_2, BLX_ALPHA):
    nn_child_1 = NeuralNetwork(nn_shape=nn_parent_1.nn_shape)
    nn_child_2 = NeuralNetwork(nn_shape=nn_parent_2.nn_shape)
    for i in range(len(nn_parent_1.weights)):
        for j in range(len(nn_parent_1.weights[i])):
            for k in range(len(nn_parent_1.weights[i][j])):
                c_min = np.minimum(nn_parent_1.weights[i][j][k], nn_parent_2.weights[i][j][k])
                c_max = np.maximum(nn_parent_1.weights[i][j][k], nn_parent_2.weights[i][j][k])
                delta_k = c_max - c_min
                nn_child_1.weights[i][j][k] = np.random.uniform(c_min - delta_k * BLX_ALPHA, c_max - delta_k * BLX_ALPHA)
                nn_child_2.weights[i][j][k] = np.random.uniform(c_min - delta_k * BLX_ALPHA, c_max - delta_k * BLX_ALPHA)

    return nn_child_1, nn_child_2

Функция для прямого распространения в ансамбле ИНС.

In [ ]:
def forward_prop_ensemble(ensemble,X):
  for count in range(len(ensemble)):
      ensemble[count].forward_propagation(X)

"Голосование" ансамбля ИНС (результат вероятность).

In [ ]:
def ensemble_bagging_preds(ensemble):
  F = 0
  for count in range(len(ensemble)):
    F += np.sum(ensemble[count].neurons[-1])/len(ensemble)
  return F

"Голосование" ансамбля ИНС (дискретный результат - 0 или 1).

In [ ]:
def ensemble_bagging(ensemble):
  F = 0
  for count in range(len(ensemble)):
    F += np.sum(ensemble[count].neurons[-1])/len(ensemble)
  if F>0.5:
    return 1
  else:
    return 0

Функция для оценивания точности обученного классификатора на тестовой выборке.

In [ ]:
def ensemble_est(ensemble,X,y):
  # размер тестового датасета
  data_set_size = y.shape[0]
  # количество правильно классифицированных случаев
  correct = 0
  for i in range(data_set_size):
    for count in range(len(ensemble)):
      population[count].forward_propagation(X.iloc[i,:].to_numpy())
    prediction = ensemble_bagging(ensemble)
    if prediction == y.iloc[i]:
      correct +=1
  accuracy = correct/data_set_size*100
  print(f'accuracy: {accuracy}%')
  return accuracy

Основной алгоритм

In [ ]:
# количество циклов эволюции
EPOCHS = 500
NN_SHAPE = (9,6,6,1)
NN_POPULATION_COUNTS = 10
BLX_ALPHA = 0.5
MUTATION_COUNTS = 2
# Обучающая выборка
N = 10
# константа корреляции
CORRELATION_CONSTANT = 0.4

# формирование начальной популяции
population = []

# количество мутаций


model_accuracy_array = []
epochs_array = []

# инициализация начальной популяции
for count in range(NN_POPULATION_COUNTS):
  neural_network = NeuralNetwork(nn_shape=NN_SHAPE)
  population.append(neural_network)

error_f = np.zeros(len(population))


In [ ]:
# получаем двух особей усечением (лучшая половина популяции)

def cut_selection():
    individual_1 = 0
    individual_2 = 0
    while individual_1 == individual_2:
        individual_1 = np.random.randint(0,NN_POPULATION_COUNTS/2)
        individual_2 = np.random.randint(0,NN_POPULATION_COUNTS/2)
    return individual_1, individual_2


In [ ]:
for epoch in range(EPOCHS):
  sort_indx = error_f.argsort() # индексы отсортированных элементов

  individual_1, individual_2 = cut_selection()
  
  # Кроссинговер BLX-a. Получаем двух потомков от двух родителей
  nn_child_1, nn_child_2 = crossbreeding(population[sort_indx[individual_1]],
                                         population[sort_indx[individual_2]],BLX_ALPHA)

  # Удаляем особи с наибольшим значением функции ошибки
  if sort_indx[-1]>sort_indx[-2]:
    population.remove(population[sort_indx[-1]])
    population.remove(population[sort_indx[-2]])
  else:
    population.remove(population[sort_indx[-2]])
    population.remove(population[sort_indx[-1]])

  population.append(nn_child_1)
  population.append(nn_child_2)

  # Гауссовская мутация
  for mut_count in range(MUTATION_COUNTS):
    mut = np.random.randint(0,len(population))
    population[mut].mutation()
  # Обнулим массив значений функции ошибок
  error_f[:]=0
  # Цикл обучения
  for learning_cycle in range(N):
    #выбираем случайный индекс элемент тренировочного датасета
    df_index = np.random.randint(0,X_train.shape[0])
    # входные данные
    piece_of_data = X_train.iloc[df_index,:].to_numpy()
    # желаемый отклик
    d = y_train.iloc[df_index]
    p = np.zeros_like(error_f)
    # прямое распространение для всей популяции
    forward_prop_ensemble(population,piece_of_data)

    # общий вывод популяции
    F = ensemble_bagging_preds(population)

    # расчёт функции корреляционного штрафа
    for count in range(len(population)):
      for count_p in range(len(population)):
        if count_p != count:
          p[count] += (population[count].neurons[-1]-F)*(population[count_p].neurons[-1]-F)
      error_f[count]+=1/N/2*(population[count].neurons[-1]-d)**2+1/N*CORRELATION_CONSTANT*p[count]
  if epoch%50==0:
    epochs_array.append(epoch)
    print(f'epoch: {epoch}')
    model_accuracy_array.append(ensemble_est(population,X_test,y_test))


Далее оценим точность классификатора на тестовой выборке.

In [ ]:
ensemble_est(population,X_test,y_test)

График обучения

In [ ]:
import matplotlib.pyplot as plt

plt.plot(epochs_array, model_accuracy_array, c='blue')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

Демонстрация вероятностного вывода.

In [ ]:
df_index = np.random.randint(0,X_test.shape[0])
piece_of_data = X_test.iloc[df_index,:].to_numpy()
d = y_test.iloc[df_index]
for count in range(len(population)):
  population[count].forward_propagation(piece_of_data)
F = ensemble_bagging_preds(population)
print(d)
print(F)

Функция для вывода весов связей в отдельный файл.

In [ ]:
def save_weights(ensemble):
  file = open('NN_ensemble_structure.txt','w')
  for nn in range(len(ensemble)):
    file.write('\n########################\n')
    for i in range(len(ensemble[nn].weights)):
      for j in range(len(ensemble[nn].weights[i])):
        for k in range(len(ensemble[nn].weights[i][j])):
          file.write(str(ensemble[nn].weights[i][j][k])+' ')
        file.write('\n')
  file.close()


In [ ]:
save_weights(population)

Функция для записи весов смещения.

In [ ]:
def save_bias(ensemble):
  file = open('NN_ensemble_bias.txt','w')
  for nn in range(len(ensemble)):
    file.write('\n########################\n')
    for i in range(len(ensemble[nn].biases)):
      for j in range(len(ensemble[nn].biases[i])):
        file.write(str(ensemble[nn].biases[i][j])+' ')
      file.write('\n')
  file.close()

In [ ]:
save_bias(population)